In [1]:
cd /home/icb/alessandro.palma/imCPA

/mnt/home/icb/alessandro.palma/imCPA


In [3]:
import pandas as pd 
import numpy as np
import metrics.metrics
import plot_utils

ModuleNotFoundError: No module named 'metrics'

In [3]:
# Load the dataset 
training_set = pd.read_csv('/home/icb/alessandro.palma/data/metadata_processed/integrated/splits/datasplit-train.csv')
validation_set = pd.read_csv('/home/icb/alessandro.palma/data/metadata_processed/integrated/splits/datasplit-val.csv')
test_set = pd.read_csv('/home/icb/alessandro.palma/data/metadata_processed/integrated/splits/datasplit-test.csv')
# OOD set is already taken from Karren 
ood_set = pd.read_csv('/home/icb/alessandro.palma/data/metadata_processed/integrated/splits/datasplit-ood.csv')

dataset = pd.concat([training_set, validation_set, test_set, ood_set], axis = 0, ignore_index = True)

In [4]:
training_set.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'SAMPLE_KEY', 'BROAD_ID', 'PLATE_ID',
       'WELL_POSITION', 'SITE', 'SAMPLE_ID', 'CPD_NAME', 'CPD_NAME_TYPE',
       'SMILES', 'STATE', 'ROW_NR_LABEL_MAT'],
      dtype='object')

In [5]:
# Load the  outlier molecules 
res = np.load('/home/icb/alessandro.palma/data/metadata_processed/original/drugs_metadata/compounds_and_embeddings.npz', 
              allow_pickle = True)

# Used if data are loaded 
mol2img_outliers=res['mol2img_outliers']

In [6]:
plotter = plot_utils.Plotter('')

In [7]:
unique_molecules = []

# Pick the outlying molecules 
for fold in mol2img_outliers:
    for sample in fold:
        # Pick unique active molecules
        mol = fold[sample][0] 
        if mol not in unique_molecules:
            unique_molecules.append(mol)

In [48]:
print(len(unique_molecules))

170


## Check overlap of such molecules with the oned from Karren

In [9]:
data_heldout = pd.read_csv('/home/icb/alessandro.palma/data/metadata/original/splits_mol2img/eval_heldout_set.csv')
data_eval_training = pd.read_csv('/home/icb/alessandro.palma/data/metadata/original/splits_mol2img/eval_in_training_set.csv')

In [10]:
held_out_compounds = np.unique(data_heldout.CPD_NAME)

In [11]:
eval_training_compounds = np.unique(data_eval_training.CPD_NAME)

In [12]:
len(np.intersect1d(unique_molecules, eval_training_compounds))

104

In [13]:
len(np.intersect1d(unique_molecules, held_out_compounds))

18

In [ ]:
# Plot unique molecules of latent outliers
latent_outliers = []
for fold in mol2img_outliers:
    for entry in fold:
        if fold[entry][0] not in plotted and np.mean(fold[entry][1][:,:,4])<0.20:
            plotter.plot_channel_panel(fold[entry][1], fold[entry][0], False, True, rgb = True)
            latent_outliers.append(fold[entry][0])
            

In [54]:
# Plot the molecules from Karren's heldout
heldout = []
for i in range(len(data_heldout)):
    mol = data_heldout.CPD_NAME[i]
    file = data_heldout.SAMPLE_KEY[i]
    if mol not in heldout:
        res = np.load(f'/storage/groups/ml01/workspace/alessandro.palma/cellpainting_512/{file}.npz')
        if np.mean(res['sample'][:,:,4]) < 50 :
            heldout.append(mol)
            res=res['sample']
            #plotter.plot_channel_panel(res, epoch=0, title=mol, save = False, plot=True, rgb=True)

In [ ]:
train_eval = []
for i in range(len(data_eval_training)):
    mol = data_eval_training.CPD_NAME[i]
    file = data_eval_training.SAMPLE_KEY[i]
    res = np.load(f'/storage/groups/ml01/workspace/alessandro.palma/cellpainting_512/{file}.npz')
    if np.mean(res['sample'][:,:,4]) < 50 :
        train_eval.append(mol)
        res=res['sample']
        plotter.plot_channel_panel(res, epoch=0, title=mol, save = False, plot=True, rgb = True)

## Check correspondence with assays 

In [32]:
from scipy.io import mmread

label_mat = mmread(f'/home/icb/alessandro.palma/data/metadata/integrated/label_matrix/label-matrix.mtx').tocsr()
col_labels = pd.read_csv(f'/home/icb/alessandro.palma/data/metadata/integrated/label_matrix/column-assay-index.csv')
row_labels = pd.read_csv(f'/home/icb/alessandro.palma/data/metadata/integrated/label_matrix/row-compound-index.csv')

In [33]:
labs = pd.DataFrame(label_mat.todense())

In [35]:
dataset

,Unnamed: 0.1,Unnamed: 0,SAMPLE_KEY,BROAD_ID,PLATE_ID,WELL_POSITION,SITE,SAMPLE_ID,CPD_NAME,CPD_NAME_TYPE,SMILES,STATE,ROW_NR_LABEL_MAT
0,161197,161197,26060-A18-3,BRD-A20314546-001-05-8,26060,A18,3,SA1861819,BRD-A20314546,BROAD_CPD_ID,O=C(COC(=O)c1nn(-c2ccccc2)c(=O)c2ccccc12)NC1CC...,trt,4524
1,162267,162267,26061-A18-5,BRD-A20314546-001-05-8,26061,A18,5,SA1861819,BRD-A20314546,BROAD_CPD_ID,O=C(COC(=O)c1nn(-c2ccccc2)c(=O)c2ccccc12)NC1CC...,trt,4524
2,160127,160127,26058-A18-1,BRD-A20314546-001-05-8,26058,A18,1,SA1861819,BRD-A20314546,BROAD_CPD_ID,O=C(COC(=O)c1nn(-c2ccccc2)c(=O)c2ccccc12)NC1CC...,trt,4524
3,165419,165419,26092-A18-1,BRD-A20314546-001-05-8,26092,A18,1,SA1861819,BRD-A20314546,BROAD_CPD_ID,O=C(COC(=O)c1nn(-c2ccccc2)c(=O)c2ccccc12)NC1CC...,trt,4524
4,161200,161200,26060-A18-6,BRD-A20314546-001-05-8,26060,A18,6,SA1861819,BRD-A20314546,BROAD_CPD_ID,O=C(COC(=O)c1nn(-c2ccccc2)c(=O)c2ccccc12)NC1CC...,trt,4524
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290197,240475,240475,25966-O16-2,BRD-K16542329-001-05-1,25966,O16,2,SA82155,zaprinast,INN,CCCOc1ccccc1-c1nc2[nH]nnc2c(=O)[nH]1,trt,392
290198,240476,240476,25966-O16-3,BRD-K16542329-001-05-1,25966,O16,3,SA82155,zaprinast,INN,CCCOc1ccccc1-c1nc2[nH]nnc2c(=O)[nH]1,trt,392
290199,240477,240477,25966-O16-4,BRD-K16542329-001-05-1,25966,O16,4,SA82155,zaprinast,INN,CCCOc1ccccc1-c1nc2[nH]nnc2c(=O)[nH]1,trt,392
290200,240478,240478,25966-O16-5,BRD-K16542329-001-05-1,25966,O16,5,SA82155,zaprinast,INN,CCCOc1ccccc1-c1nc2[nH]nnc2c(=O)[nH]1,trt,392


In [36]:
ds_unique = dataset.drop_duplicates(subset='CPD_NAME')

In [37]:
ds_unique.index = np.arange(len(ds_unique))

In [38]:
ds_unique['ACT_ASSAYS'] = [0 for i in range(len(ds_unique))]
ds_unique

<ipython-input-38-7c6b4fdfedaf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_unique['ACT_ASSAYS'] = [0 for i in range(len(ds_unique))]


,Unnamed: 0.1,Unnamed: 0,SAMPLE_KEY,BROAD_ID,PLATE_ID,WELL_POSITION,SITE,SAMPLE_ID,CPD_NAME,CPD_NAME_TYPE,SMILES,STATE,ROW_NR_LABEL_MAT,ACT_ASSAYS
0,161197,161197,26060-A18-3,BRD-A20314546-001-05-8,26060,A18,3,SA1861819,BRD-A20314546,BROAD_CPD_ID,O=C(COC(=O)c1nn(-c2ccccc2)c(=O)c2ccccc12)NC1CC...,trt,4524,0
1,38536,38536,25378-J20-6,BRD-K01868680-001-05-0,25378,J20,6,SA1860458,BRD-K01868680,BROAD_CPD_ID,C1COCCOCCOc2ccccc2OCCOCCOCCOc2ccccc2OCCOCCO1,trt,6473,0
2,158872,158872,25991-O24-6,BRD-K77133231-001-01-1,25991,O24,6,SA63225,PD 169316,common,[O-][N+](=O)c1ccc(cc1)-c1nc(c([nH]1)-c1ccncc1)...,trt,254,0
3,12846,12846,24307-A09-1,BRD-A98378129-001-01-4,24307,A09,1,SA792902,talniflumate,to-be-curated,FC(F)(F)c1cccc(Nc2ncccc2C(=O)OC2OC(=O)c3ccccc2...,trt,1316,0
4,27947,27947,24618-D07-6,BRD-K84797651-001-01-5,24618,D07,6,SA849437,BRD-K84797651,BROAD_CPD_ID,C[C@@H](O)C#Cc1ccc2c(O[C@@H](CN(C)Cc3ccc(cc3)-...,trt,9932,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10556,201837,201837,24305-M11-1,BRD-K14200658-001-02-6,24305,M11,1,SA83079,syrosingopine,INN,CCOC(=O)Oc1c(OC)cc(cc1OC)C(=O)O[C@@H]1C[C@@H]2...,trt,3987,0
10557,201717,201717,24305-G15-1,BRD-K08078237-001-04-0,24305,G15,1,SA58539,tetrandrine,primary-common,COc1ccc2C[C@@H]3N(C)CCc4cc(OC)c(OC)c(Oc5cc6[C@...,trt,929,0
10558,198987,198987,24277-P20-1,BRD-K40255344-001-05-1,24277,P20,1,SA82361,tyrphostin A9,primary-common,CC(C)(C)c1cc(C=C(C#N)C#N)cc(c1O)C(C)(C)C,trt,671,0
10559,201735,201735,24305-H02-1,BRD-K08356259-003-05-6,24305,H02,1,SA83160,xylometazoline,INN,Cc1cc(cc(C)c1CC1=NCCN1)C(C)(C)C,trt,647,0


In [39]:
# Collect dataframe of labels 
drug2ones = {}
for i in range(len(ds_unique)):
    drug = ds_unique.CPD_NAME[i]
    idx = ds_unique.ROW_NR_LABEL_MAT[i]
    ones = np.sum(label_mat.todense()[idx]==1)
    drug2ones[drug] = ones
    ds_unique.at[i, 'ACT_ASSAYS'] = ones


In [40]:
# Sort dict 
sorted_items = sorted(drug2ones.items(), key=lambda item: item[1], reverse = True)
top_10_perc = sorted_items[:1000]
print(len(sorted_items))
top_10_perc = [comp[0] for comp in top_10_perc]
len(set(top_10_perc).intersection(set(unique_molecules)))
len(set(top_10_perc).intersection(set(held_out_compounds)))
len(set(top_10_perc).intersection(set(eval_training_compounds)))


10561


288

In [ ]:
sorted_items

In [82]:
ds_sorted = ds_unique.sort_values(by='ACT_ASSAYS', ascending = False)[0:200]
ds_sorted

,Unnamed: 0.1,Unnamed: 0,SAMPLE_KEY,BROAD_ID,PLATE_ID,WELL_POSITION,SITE,SAMPLE_ID,CPD_NAME,CPD_NAME_TYPE,SMILES,STATE,ROW_NR_LABEL_MAT,ACT_ASSAYS
9320,146035,146035,25965-A23-3,BRD-K00259736-001-03-2,25965,A23,3,SA38604,colchicine,JAN,COc1cc2CC[C@H](NC(C)=O)c3cc(=O)c(OC)ccc3-c2c(O...,trt,208,80
10536,198651,198651,24277-C02-1,BRD-K21680192-300-05-2,24277,C02,1,SA83157,mitoxantrone,INN,OCCNCCNc1ccc(NCCNCCO)c2C(=O)c3c(O)ccc(O)c3C(=O...,trt,100,77
10558,198987,198987,24277-P20-1,BRD-K40255344-001-05-1,24277,P20,1,SA82361,tyrphostin A9,primary-common,CC(C)(C)c1cc(C=C(C#N)C#N)cc(c1O)C(C)(C)C,trt,671,77
10553,201729,201729,24305-H01-1,BRD-K08316444-001-02-7,24305,H01,1,SA58038,rotenone,primary-common,COc1cc2OC[C@H]3Oc4c5C[C@@H](Oc5ccc4C(=O)[C@H]3...,trt,75,77
10547,203457,203457,24310-F21-1,BRD-K47869605-001-04-9,24310,F21,1,SA58786,podophyllotoxin,BAN,COc1cc(cc(OC)c1OC)[C@H]1[C@@H]2[C@H](COC2=O)[C...,trt,105,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10288,223422,223422,25584-A05-1,BRD-A86381030-001-10-1,25584,A05,1,SA1859817,BRD-A86381030,BROAD_CPD_ID,Oc1c(cc(Cl)c2cccnc12)C(Nc1ccccn1)c1cccnc1,trt,6701,9
10413,222264,222264,25579-G05-1,BRD-K67649668-001-05-1,25579,G05,1,SA1857943,BRD-K67649668,BROAD_CPD_ID,Cc1ccc(cc1)S(=O)(=O)C(\NC(=O)c1ccccc1)=C\Cl,trt,8061,9
6962,168556,168556,26115-A16-6,BRD-K48496310-001-05-3,26115,A16,6,SA1861164,BRD-K48496310,BROAD_CPD_ID,CS(=O)(=O)c1ccc(cc1[N+]([O-])=O)C(=O)NCCN1C(=O...,trt,8011,9
6336,100620,100620,25605-E22-3,BRD-K25369035-001-06-1,25605,E22,3,SA1854368,BRD-K25369035,BROAD_CPD_ID,CC(C)Cn1c(nc(c1S(=O)(=O)c1ccc(F)cc1)[N+]([O-])...,trt,2610,9


In [83]:
label_based = []
for i in range(len(ds_sorted)):
    mol = data_eval_training.CPD_NAME[i]
    file = data_eval_training.SAMPLE_KEY[i]
    res = np.load(f'/storage/groups/ml01/workspace/alessandro.palma/cellpainting_512/{file}.npz')
    res=res['sample']
    if np.mean(res[:,:,4])<50:
        label_based.append(mol)
        #plotter.plot_channel_panel(res, epoch=0, title=mol, save = False, plot=True)

## Fuse the molecules from the three sources

In [87]:
subset = list(set(heldout).union(train_eval).union(ds_sorted.CPD_NAME)) + ['DMSO']

In [91]:
len(subset)

1639

In [92]:
dataset_filter = dataset.query('CPD_NAME in @subset')

In [94]:
np.unique(dataset_filter['STATE'], return_counts = True)

(array(['ctr', 'trt'], dtype=object), array([ 6168, 52287]))

In [99]:
training_set_filter = training_set.query('CPD_NAME in @subset')
validation_set_filter = validation_set.query('CPD_NAME in @subset')
test_set_filter = test_set.query('CPD_NAME in @subset')
ood_set_filter = ood_set.query('CPD_NAME in @subset')

In [101]:
len(test_set_filter)

10170

In [106]:
# Save the downsized splits 
training_set_filter.to_csv(f'/home/icb/alessandro.palma/data/metadata_processed/integrated/50k/splits/datasplit-train.csv')
test_set_filter.to_csv(f'/home/icb/alessandro.palma/data/metadata_processed/integrated/50k/splits/datasplit-test.csv')
validation_set_filter.to_csv(f'/home/icb/alessandro.palma/data/metadata_processed/integrated/50k/splits/datasplit-val.csv')
ood_set_filter.to_csv(f'/home/icb/alessandro.palma/data/metadata_processed/integrated/50k/splits/datasplit-ood.csv')

In [112]:
ds = np.load('/home/icb/alessandro.palma/data/data_index/integrated/50k/train_data_index.npz', allow_pickle=True)

In [117]:
np.sum(ds['state']=='ctr')

4319

In [17]:
a = pd.read_csv('/home/icb/alessandro.palma/data/metadata_processed/original/embeddings/rdkit/rdkit2D_embedding_cellpainting.csv', index_col=0).loc['COc1cc-2c(C[C@@H]3N(C)CCc4cc(O)c(OC)c-2c34)cc1O']

KeyError: 'COc1cc-2c(C[C@@H]3N(C)CCc4cc(O)c(OC)c-2c34)cc1O'

In [ ]:
np.array(a)